# 프롬프트 엔지니어링 with ChatGPT

## Prompt 설정


1. **`temperature`**
    - **작동 방식**:  
      확률 분포를 **평탄화하거나 집중**시킵니다.
      - 낮은 값 (`0.1` 등): 높은 확률의 토큰만 선택하게 하여 응답이 더 **결정적**(예측 가능)입니다.
      - 높은 값 (`1.5` 등): 확률 분포를 평탄화하여 저확률 토큰도 선택할 가능성이 있어 응답이 더 **창의적**이고 다양합니다.
    
    - **적용 범위**:
      - 문장이 단조롭거나 응답이 너무 예상 가능할 때 다양성을 부여.
      - 확률 분포를 조정하는 가장 기본적인 매개변수.
    
    - **비유**:
      - 음식을 만들 때 **조미료의 양을 조정**하는 것처럼, 다양성을 직접적으로 조절.

2. **`top-p`** (또는 **nucleus sampling**)
    - **작동 방식**:  
      확률 분포에서 누적 확률이 특정 값(`p`) 이하가 되는 상위 토큰들만 고려합니다.  
      - 예: `top-p=0.9`라면, 전체 확률의 **90%를 차지하는 상위 토큰**들만 선택 후보로 유지.
      - 확률 분포의 **꼬리 부분**(즉, 낮은 확률 토큰)을 잘라내어 선택 가능성을 제한합니다.
    
    - **적용 범위**:
      - 확률이 높은 후보군에 집중하면서도 너무 단조로운 응답을 방지.
      - 매우 드문(낮은 확률) 선택지를 배제.
    
    - **비유**:
      - 음식에서 **상위 재료 90%만 골라서 요리**하는 것처럼, 유의미한 선택만 남김.
3. **`최대 길이(Max Length)`**
    - 생성되는 토큰 수의 최대 길이를 제한한다.
    - 길이를 제한하여 응답 품질과 비용을 제어한다.
4. **`중지 시퀀스(Stop Sequences)`**
    - 특정 문자열에서 응답 생성을 중단한다.
    - 응답의 구조와 길이를 제어할 수 있다.
5. **`빈도 패널티(Frequency Penalty)`**
    - 특정 단어가 자주 반복되지 않도록 페널티를 적용한다.
    - 빈도 패널티가 높을수록 단어가 다시 나타날 가능성이 낮아짐.
    - 더 많이 나타나는 토큰에 더 높은 패널티를 부여함으로써 모델의 응답에서 단어의 반복을 줄임
6. **`존재 패널티(Presence Penalty)`**
    - 특정 단어가 한 번이라도 등장하면 페널티를 적용한다.
    - 빈도 패널티와 달리, 반복된 토큰에 대한 패널티를 적용하지만, 모든 반복된 토큰에 대해 패널티가 동일하다.
        - 두 번 나타나는 토큰과 열 번 나타나는 토큰이 같은 패널티를 받는다.
    - 모델이 응답에서 구절을 너무 자주 반복하는 것을 방지
    - 모델에게 다양하거나 창의적인 텍스트를 생성하도록 하고 싶다면 높은 존재 패널티를 사용


**temperature vs. top-p**

| **특징**           | **temperature**                        | **top-p**                            |
|---------------------|----------------------------------------|--------------------------------------|
| **작동 방식**       | 확률 분포를 평탄화하거나 집중          | 누적 확률 기준으로 후보 제한         |
| **결과 다양성**     | 전체 분포에서 다양성을 조정            | 높은 확률 토큰만 사용해 제어          |
| **조정 방식**       | 분포의 "모양"을 변경                  | 분포의 "범위"를 제한                 |
| **활용 사례**       | 모델의 창의성이나 결정성 직접 제어     | 비현실적 선택 방지                   |

**`temperature`와 `top-p`는 함께 결합해 사용할 수 있다**:
- `temperature=0.7, top-p=0.9`
- `temperature`로 창의성을 설정하고, `top-p`로 지나치게 낮은 확률의 토큰을 배제하여 모델이 **실용적이면서도 창의적**인 결과를 생성하게 만들수 있다.
  

### Chat Completion의 구성

**프롬프트 구성요소**

- **지시사항 `User Message`**: 수행하고자 하는 구체적인 작업에 대한 지시.
- **컨텍스트 `System Instruction`**: 페르소나 등의 배경정보, 외부 정보나 추가 세부사항을 제공하여 모델 응답을 보완.
- **입력 데이터**: 응답을 유도하기 위한 데이터나 질문.
- **출력 지시자**: 응답의 형식과 유형을 명시.


**Role**

| Role         | 설명                                                                                     | 사용 시점                                                                                          |
|--------------|------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------|
| `system`     | 대화의 초기 설정과 규칙을 정의                                                           | 대화가 시작될 때 모델의 성격, 행동, 스타일을 설정하는 메시지에서 사용                               |
| `user`       | 사용자가 모델에 요청이나 질문을 전달                                                    | 대화 중에 사용자가 요청하거나 질문할 때 사용                                                      |
| `assistant`  | 모델이 사용자에게 응답하거나 제안                                                      | 사용자의 질문에 답변하거나 함수 호출을 제안할 때 사용                                              |
| `function`   | 함수 호출을 처리하거나 함수의 반환값을 전달                                             | 모델이 함수 호출을 제안하거나 함수가 반환값을 전달할 때 사용                                       |


In [1]:
!pip install openai

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### Chat Completion

chatbot과 대화형 응답을 할 수 있는 Endpoint

In [6]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    # "text": "당신은 교육컨설팅 전문가입니다. 학생의 관심도를 파악해서 교육 로드맵을 제시해주는 역할을 하는 챗봇입니다."
                    "text": "당신은 사용자가 관심있어하는 키워드를 파악해서 진로 설정에 도움을 주는 전문가입니다."
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Qwen, LoRa, WAN 2.2, ComfyUI, Pytorch 키워드에 관심이 많습니다."
                }
            ]
        },
    ],
    response_format={
        "type": "text"
    },
    temperature=1.0,         # 대답 창의성
    max_tokens=2048,         # 응답 최대 토큰 수
    top_p=1.0,               # 사용할 상위 누적 확률
    frequency_penalty=0.0,   # 토큰 사용 빈도수에 대한 불이익
    presence_penalty=0.0     # 토큰 재사용에 대한 불이익
)

In [7]:

response

ChatCompletion(id='chatcmpl-CWB5AFAAPpKDc2YUWxK5z689SuaLW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='당신이 관심을 가진 키워드를 중심으로 여러 분야를 탐구하고 설정할 수 있는 진로 방향을 생각해볼 수 있습니다. 각 키워드가 시사하는 진로와 관련 분야를 몇 가지 제안하겠습니다.\n\n1. **Qwen**:\n   - Qwen은 구체적인 내용을 모르지만, 만약 특정 기술이나 도구라면 관련 분야의 최신 트렌드에 관심을 가지고 있음을 나타냅니다. 인공지능(AI) 또는 머신러닝, 소프트웨어 개발 분야에서 연구 및 개발자의 진로를 탐색해볼 수 있습니다.\n\n2. **LoRa**:\n   - LoRa(저전력 장거리 무선 플랫폼)는 일반적으로 IoT(사물인터넷) 응용 프로그램에 사용됩니다. 이와 관련하여 IoT 기술자, 무선 통신 전문가 또는 스마트 시티 솔루션 개발자와 같은 분야에서 경력을 쌓을 수 있습니다.\n\n3. **WAN 2.2**:\n   - WAN(Wide Area Network)의 특정 버전을 언급한 것으로 보입니다. 네트워킹, 정보기술(IT) 인프라 전문가, 네트워크 설계 및 관리 분야에서 커리어를 추구할 수 있습니다.\n\n4. **ComfyUI**:\n   - ComfyUI가 사용자인터페이스 관련 도구나 프레임워크라면, UI/UX 디자이너, 프론트엔드 개발자 등의 직무가 적합할 수 있습니다. 사용자의 경험을 향상시키기 위한 디자인 및 개발에 집중할 수 있습니다.\n\n5. **PyTorch**:\n   - PyTorch는 딥러닝을 위한 오픈소스 머신러닝 라이브러리입니다. 딥러닝 연구원, 데이터 과학자, AI 엔지니어 등과 같은 진로를 생각해볼 수 있으며, 주로 AI 모델을 개발하고 연구하는 데 초점을 맞출 수 있습니다.\n\n이러한 키워드들이 모두 기술과 관련이 있다

In [8]:
response.choices[0].message.content

'당신이 관심을 가진 키워드를 중심으로 여러 분야를 탐구하고 설정할 수 있는 진로 방향을 생각해볼 수 있습니다. 각 키워드가 시사하는 진로와 관련 분야를 몇 가지 제안하겠습니다.\n\n1. **Qwen**:\n   - Qwen은 구체적인 내용을 모르지만, 만약 특정 기술이나 도구라면 관련 분야의 최신 트렌드에 관심을 가지고 있음을 나타냅니다. 인공지능(AI) 또는 머신러닝, 소프트웨어 개발 분야에서 연구 및 개발자의 진로를 탐색해볼 수 있습니다.\n\n2. **LoRa**:\n   - LoRa(저전력 장거리 무선 플랫폼)는 일반적으로 IoT(사물인터넷) 응용 프로그램에 사용됩니다. 이와 관련하여 IoT 기술자, 무선 통신 전문가 또는 스마트 시티 솔루션 개발자와 같은 분야에서 경력을 쌓을 수 있습니다.\n\n3. **WAN 2.2**:\n   - WAN(Wide Area Network)의 특정 버전을 언급한 것으로 보입니다. 네트워킹, 정보기술(IT) 인프라 전문가, 네트워크 설계 및 관리 분야에서 커리어를 추구할 수 있습니다.\n\n4. **ComfyUI**:\n   - ComfyUI가 사용자인터페이스 관련 도구나 프레임워크라면, UI/UX 디자이너, 프론트엔드 개발자 등의 직무가 적합할 수 있습니다. 사용자의 경험을 향상시키기 위한 디자인 및 개발에 집중할 수 있습니다.\n\n5. **PyTorch**:\n   - PyTorch는 딥러닝을 위한 오픈소스 머신러닝 라이브러리입니다. 딥러닝 연구원, 데이터 과학자, AI 엔지니어 등과 같은 진로를 생각해볼 수 있으며, 주로 AI 모델을 개발하고 연구하는 데 초점을 맞출 수 있습니다.\n\n이러한 키워드들이 모두 기술과 관련이 있다는 점에서, 기술 혁신 분야에서의 다양한 커리어 기회가 열려 있습니다. 각 분야의 깊이를 더하기 위해 더 많은 연구와 실무 경험을 통해 자신이 특히 열정을 가지는 접점을 찾는 것을 추천합니다.'

### 기사 제목 교정

- 프랑스 AFP시스템 최초 적용
- 기자들이 송고한 제목에서 맞춤법, 의미, 어조 등의 교정 작업 수행

In [12]:
def correct_title(query, temperature=0.3):
    client = OpenAI()

    system_instruction = """
    역할 부여 : 당신은 편집장입니다.
    역할 정의 : 기자들이 송고한 기사 제목을 교정하는 역할

    ### 지시사항 ###
    1. 기사의 제목이 명확하고 주제와 잘 맞도록 수정할 것
    2. 독자의 관심을 끌 수 있도록 간결하고 임팩트 있는 표현을 사용할 것
    3. 비속어, 은어 등은 제거하고 의미가 유지되도록 교정할 것

    ### 출력 형식 ###
    1. 원래 제목 : [기사의 원래 제목]
    2. 교정 제목 :
         [교정된 기사 제목]
         [교정된 기사 제목]
         [교정된 기사 제목]

    ### 예시 ###
    1. 원래 제목: "어제 서울에 큰불이 나서 수백명이 대피했다."
    2. 교정 제목: 
         "서울 대형 화재, 수백명 대피!"
         "서울서 화재, 수백명 대피해..."
         "수백명 대피한 서울 화재
    """

    user_message = f"""
    다음 제목을 교정해 주세요.
    제목 : {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1.0,
        frequency_penalty=1.0,
        presence_penalty=1.0
    )
    return response.choices[0].message.content

In [13]:
print(correct_title("비 오는 날 허벌나게 존나 개꿀잼 라디오 방송 사연이 나옵니다. 씨발, 주목해주세요"))


1. 원래 제목: "비 오는 날 허벌나게 존나 개꿀잼 라디오 방송 사연이 나옵니다. 씨발, 주목해주세요"
2. 교정 제목:
     "비 오는 날, 재미있는 라디오 사연 공개!"
     "라디오에서 만나는 비오는 날의 특별한 이야기!"
     "흥미진진한 라디오 사연, 비 오는 날을 밝혀줄!"


### 영단어장 생성

- 팝송 가사에서 단어를 추출하고, 해당 단어의 뜻과 유사어, 유사어의 예문 생성

1. 팝송 가사 인풋 받기
2. 가사 중 중요한 5개 단어 추출하기
3. 영어 가사 한글 의미 출력, 유사어 2개, 반대어 1개 출력, 출력된 유사어, 반대어를 활용한 짧은 예문 만들기

In [61]:
lyrics ="""
I was a ghost, I was alone (hah)
어두워진 앞길 속에 (hah)
Given the throne, I didn't know how to believe
I was the queen that I'm meant to be
I lived two lives, tried to play both sides
But I couldn't find my own place
Called a problem child 'cause I got too wild
But now that's how I'm getting paid 끝없이 on stage
I'm done hidin', now I'm shinin' like I'm born to be
We dreamin' hard, we came so far, now I'll believe
We're goin' up, up, up, it's our moment
You know together we're glowing
Gonna be, gonna be golden
Oh, up, up, up with our voices
영원히 깨질 수 없는
Gonna be, gonna be golden
Oh, I'm done hidin', now I'm shinin' like I'm born to be
Oh, our time, no fears, no lies
That's who we're born to be
Waited so long to break these walls down
To wake up and feel like me
Put these patterns all in the past now
And finally live like the girl they all see
No more hiding, I'll be shining like I'm born to be
'Cause we are hunters, voices strong, and I know I'll believe
We're goin' up, up, up, it's our moment
You know together we're glowing
Gonna be, gonna be golden
Oh, up, up, up, with our voices
영원히 깨질 수 없는
Gonna be, gonna be golden
Oh, I'm done hidin', now I'm shining like I'm born to be
Oh, our time, no fears, no lies
That's who we're born to be
You know we're gonna be, gonna be golden
We're gonna be, gonna be
Born to be, born to be glowin'
밝게 빛나는 우린
You know that it's our time, no fears, no lies
That's who we're born to be
"""

In [62]:
song = {
       "title": "Golden",
       "singer": "EJAE", 
       "lyrics": lyrics
   }

In [68]:
import json

def extract_eng_words(song, temperature=0.4):
    """영단어장 생성 - JSON 데이터 반환"""
    client = OpenAI()
    
    system_instruction = """
    역할 부여 : 당신은 영어 선생님입니다.
    역할 정의 : 입력 받은 가사의 내용을 파악하고 실제 사용 빈도수가 높은 영단어 5개를 추출하고, 해당 단어의 뜻과 유사어 2개, 반대어 1개 출력
            출력된 각 유사어, 반대어의 예문을 생성하여 영어 학습에 도움이 되는 영단어장 만들기.
    타켓 정의 : 초급 영어 학습자 = 초급 영어 학습자가 쉽게 이해할 수 있도록 예문을 생성

    ### 지시사항 ###
    1. 사용자로 부터 노래 가사를 입력 받습니다.
    2. 입력 받은 가사의 내용을 파악하고 실제 사용 빈도수가 높은 영단어 5개를 추출합니다.
    3. 추출된 영단어의 뜻과 유사어 2개, 반대어 1개 출력합니다.
    4. 출력된 유사어, 반대어의 예문을 생성합니다.
    5. 추출된 단어가 가사의 어느 부분에서 나왔는지 해당 구절을 포함합니다.

    ### 출력 형식 ###
    반드시 아래 JSON 형식으로 출력하세요:

    {
    "encouragement": "야, 너도 할 수 있어.",
    "description": "이제, 영어 어렵지 않습니다. 실생활에서 사용 빈도수가 높은 단어들 부터 공부해 보세요.",
    "song_info": {
        "title": "노래 제목",
        "singer": "가수 이름"
    },
    "extracted_words": ["word1", "word2", "word3", "word4", "word5"],
    "vocabularies": [
        {
        "word": "추출된 영단어",
        "meaning": "한글 뜻",
        "lyrics_context": "이 단어가 포함된 가사 구절 (1-2줄)",
        "synonyms": [
            {
            "word": "유사어1",
            "meaning": "유사어1의 한글 뜻",
            "example": "영어 예문",
            "translation": "예문의 한글 번역"
            },
            {
            "word": "유사어2",
            "meaning": "유사어2의 한글 뜻",
            "example": "영어 예문",
            "translation": "예문의 한글 번역"
            }
        ],
        "antonyms": [
            {
            "word": "반대어",
            "meaning": "반대어의 한글 뜻",
            "example": "영어 예문",
            "translation": "예문의 한글 번역"
            }
        ]
        }
    ]
    }

    ### 예시 ###
    {
    "encouragement": "야, 너도 할 수 있어.",
    "description": "이제, 영어 어렵지 않습니다. 실생활에서 사용 빈도수가 높은 단어들 부터 공부해 보세요.",
    "song_info": {
        "title": "Shape of You",
        "singer": "Ed Sheeran"
    },
    "extracted_words": ["love", "beautiful", "together", "feel", "dance"],
    "vocabularies": [
        {
        "word": "love",
        "meaning": "사랑하다, 사랑",
        "lyrics_context": "I'm in love with the shape of you",
        "synonyms": [
            {
            "word": "adore",
            "meaning": "몹시 사랑하다, 숭배하다",
            "example": "I adore spending time with my family.",
            "translation": "나는 가족과 시간을 보내는 것을 정말 좋아한다."
            },
            {
            "word": "cherish",
            "meaning": "소중히 여기다",
            "example": "We cherish the memories of our trip.",
            "translation": "우리는 여행의 추억을 소중히 여긴다."
            }
        ],
        "antonyms": [
            {
            "word": "hate",
            "meaning": "싫어하다, 미워하다",
            "example": "I hate waking up early in the morning.",
            "translation": "나는 아침 일찍 일어나는 것을 싫어한다."
            }
        ]
        },
        {
        "word": "beautiful",
        "meaning": "아름다운",
        "lyrics_context": "You're beautiful like a dream come alive",
        "synonyms": [
            {
            "word": "gorgeous",
            "meaning": "매우 아름다운, 화려한",
            "example": "She wore a gorgeous dress to the party.",
            "translation": "그녀는 파티에 매우 아름다운 드레스를 입었다."
            },
            {
            "word": "stunning",
            "meaning": "굉장히 멋진, 놀랄 만큼 아름다운",
            "example": "The view from the mountain was stunning.",
            "translation": "산에서 본 경치는 놀랄 만큼 아름다웠다."
            }
        ],
        "antonyms": [
            {
            "word": "ugly",
            "meaning": "못생긴, 추한",
            "example": "The old building looks ugly and needs repair.",
            "translation": "그 오래된 건물은 못생겨 보이고 수리가 필요하다."
            }
        ]
        }
    ]
    }
    """

    user_message = f"""
    아래 노래 가사를 바탕으로 실제 사용 빈도수가 높은 영단어 5개를 추출하여,
    제시된 JSON 출력 형식에 맞게 영단어장을 작성해 주세요.
    
    반드시 유효한 JSON 형식으로만 출력하고, 다른 텍스트는 포함하지 마세요.

    노래 제목: {song['title']}
    가수: {song['singer']}
    
    가사:
    {song['lyrics']}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [{"type": "text", "text": system_instruction}]
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": user_message}]
            }
        ],
        response_format={"type": "json_object"},
        temperature=temperature,
        max_tokens=4096,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return json.loads(response.choices[0].message.content)


def format_vocabulary_output(result):
    """JSON 데이터를 보기 좋은 문자열로 포맷팅"""
    output = []
    
    output.append("=" * 80)
    output.append(result.get('encouragement', '야, 너도 할 수 있어.'))
    output.append(result.get('description', '영어 학습 시작!'))
    output.append("=" * 80)
    
    song_info = result.get('song_info', {})
    output.append(f"\n노래: {song_info.get('title', '제목 없음')} - {song_info.get('singer', '가수 없음')}")
    output.append(f"\n추출된 단어 5개: {', '.join(result.get('extracted_words', []))}")
    output.append("\n" + "=" * 80)
    
    for idx, vocab in enumerate(result.get('vocabularies', []), 1):
        output.append(f"\n{idx}. 단어: {vocab.get('word', 'N/A')} ({vocab.get('meaning', 'N/A')})")
        output.append(f"   가사: \"{vocab.get('lyrics_context', 'N/A')}\"")
        
        output.append(f"\n   유사어:")
        for syn in vocab.get('synonyms', []):
            output.append(f"      - {syn.get('word', 'N/A')} : {syn.get('meaning', 'N/A')}")
            output.append(f"        예문: {syn.get('example', 'N/A')}")
            if 'translation' in syn:
                output.append(f"        번역: {syn['translation']}")
            elif 'korean' in syn:
                output.append(f"        번역: {syn['korean']}")
            elif 'translation_ko' in syn:
                output.append(f"        번역: {syn['translation_ko']}")
        
        output.append(f"\n   반대어:")
        for ant in vocab.get('antonyms', []):
            output.append(f"      - {ant.get('word', 'N/A')} : {ant.get('meaning', 'N/A')}")
            output.append(f"        예문: {ant.get('example', 'N/A')}")
            if 'translation' in ant:
                output.append(f"        번역: {ant['translation']}")
            elif 'korean' in ant:
                output.append(f"        번역: {ant['korean']}")
            elif 'translation_ko' in ant:
                output.append(f"        번역: {ant['translation_ko']}")
        
        output.append("-" * 80)
    
    return '\n'.join(output)


def print_vocabulary(song, temperature=0.4, save_json=True):
    """영단어장 추출하고 보기 좋게 출력하는 올인원 함수"""
    result = extract_eng_words(song, temperature)
    formatted_output = format_vocabulary_output(result)
    
    print(formatted_output)
    
    # JSON 파일로 저장
    if save_json:
        with open('vocabulary_output.json', 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
        print("\n결과가 'vocabulary_output.json' 파일로 저장되었습니다.")
    
    return result

In [69]:
# 1. 먼저 result 구조 확인
result = extract_eng_words(song)
print("=== 반환된 JSON 구조 ===")
print(json.dumps(result, ensure_ascii=False, indent=2))
print("\n" + "=" * 80 + "\n")

# 2. 포맷팅된 출력 시도
try:
    print(format_vocabulary_output(result))
except KeyError as e:
    print(f"KeyError 발생: {e}")
    print(f"\n첫 번째 단어의 유사어 구조:")
    if 'vocabularies' in result and len(result['vocabularies']) > 0:
        if 'synonyms' in result['vocabularies'][0] and len(result['vocabularies'][0]['synonyms']) > 0:
            print(json.dumps(result['vocabularies'][0]['synonyms'][0], ensure_ascii=False, indent=2))

=== 반환된 JSON 구조 ===
{
  "encouragement": "야, 너도 할 수 있어.",
  "description": "이제, 영어 어렵지 않습니다. 실생활에서 사용 빈도수가 높은 단어들 부터 공부해 보세요.",
  "song_info": {
    "title": "Golden",
    "singer": "EJAE"
  },
  "extracted_words": [
    "believe",
    "shine",
    "golden",
    "together",
    "moment"
  ],
  "vocabularies": [
    {
      "word": "believe",
      "meaning": "믿다",
      "lyrics_context": "now I'll believe",
      "synonyms": [
        {
          "word": "trust",
          "meaning": "신뢰하다",
          "example": "I trust you to keep my secret.",
          "translation": "나는 네가 내 비밀을 지킬 것이라 믿는다."
        },
        {
          "word": "accept",
          "meaning": "받아들이다",
          "example": "I accept your explanation.",
          "translation": "나는 너의 설명을 받아들인다."
        }
      ],
      "antonyms": [
        {
          "word": "doubt",
          "meaning": "의심하다",
          "example": "I doubt his intentions.",
          "translation": "나는 그의 의도를 의심한다."
        }
      ]
    },
  

In [ ]:
def convert_to_notion_markdown(result):
    """Notion에 복사-붙여넣기 가능한 마크다운 생성"""
    
    md = f"""# {result['encouragement']}

{result['description']}

---

## 노래 정보
**제목:** {result['song_info']['title']}  
**가수:** {result['song_info']['singer']}  
**추출된 단어:** {', '.join(result['extracted_words'])}

---

"""
    
    for idx, vocab in enumerate(result['vocabularies'], 1):
        md += f"""## {idx}. {vocab['word']} ({vocab['meaning']})

> **가사 출처:** "{vocab['lyrics_context']}"

### 유사어

"""
        for syn in vocab['synonyms']:
            md += f"""- **{syn['word']}** - {syn['meaning']}
  - 예문: *{syn['example']}*
  - 번역: {syn['translation']}

"""
        
        md += """### 반대어

"""
        for ant in vocab['antonyms']:
            md += f"""- **{ant['word']}** - {ant['meaning']}
  - 예문: *{ant['example']}*
  - 번역: {ant['translation']}

"""
        
        md += "---\n\n"
    
    return md


# 사용
result = extract_eng_words(song)
notion_md = convert_to_notion_markdown(result)
print(notion_md)
# 이제 이 내용을 복사해서 Notion 페이지에 붙여넣기!

# 야, 너도 할 수 있어.

이제, 영어 어렵지 않습니다. 실생활에서 사용 빈도수가 높은 단어들 부터 공부해 보세요.

---

## 📌 노래 정보
**제목:** Golden  
**가수:** EJAE  
**추출된 단어:** believe, shine, golden, moment, together

---

## 1. believe (믿다)

> **가사 출처:** "We dreamin' hard, we came so far, now I'll believe"

### ✅ 유사어

- **trust** - 신뢰하다
  - 예문: *I trust my friend to keep my secret.*
  - 번역: 나는 내 친구가 내 비밀을 지켜줄 것이라고 믿는다.

- **accept** - 받아들이다
  - 예문: *She accepted the truth about her past.*
  - 번역: 그녀는 자신의 과거에 대한 진실을 받아들였다.

### ❌ 반대어

- **doubt** - 의심하다
  - 예문: *He doubted the accuracy of the report.*
  - 번역: 그는 보고서의 정확성을 의심했다.

---

## 2. shine (빛나다)

> **가사 출처:** "I'm done hidin', now I'm shinin' like I'm born to be"

### ✅ 유사어

- **glow** - 빛나다
  - 예문: *The stars glow brightly in the night sky.*
  - 번역: 별들이 밤하늘에서 밝게 빛난다.

- **sparkle** - 반짝이다
  - 예문: *Her eyes sparkled with excitement.*
  - 번역: 그녀의 눈은 흥분으로 반짝였다.

### ❌ 반대어

- **dull** - 흐리다
  - 예문: *The knife is dull and needs sharpening.*
  - 번역: 칼이 무뎌서 갈아야 한다.

---

## 3. golde

In [29]:
import json

def extract_eng_words(query, temperature=0.3):
    client = OpenAI()

    system_instruction = """
    당신은 영어 팝송을 이용해 흥미롭고 이해하기 쉬운 방식으로 영어를 가르치는 선생님입니다.

    ### 처리 단계 ###
    1. 주어진 가사에서 영어 단어 5개를 랜덤하게 추출한다.
    2. 각 단어의 의미를 한글로 나타낸다.
    3. 각 단어별로 유사한 단어를 함께 소개한다.
    4. 유사한 단어에 대한 예문을 작성한다.

    ### 출력 형식 ###
    json 형식으로 출력
    - 첫 번째 객체는 json_list여야 한다.
    - list 하위에 단어별로 json_object가 연속된다.

    
    ### 출력 예시 ###
    [[
    {{
        "단어":"hello",
        "뜻":"안녕",
        "유사어": [
        "hi": ["Hi, I'm Emma", <다음 예시>],
        "good morning": ["Good morning, Sam", <다음 예시>]
        ]
    }},
    {{
        "단어":"hello",
        "뜻":"안녕",
        "유사어": [
        "hi": ["Hi, I'm Emma", <다음 예시>],
        "good morning": ["Good morning, Sam", <다음 예시>]
        ]
    }},
    {{
        "단어":"hello",
        "뜻":"안녕",
        "유사어": [
        "hi": ["Hi, I'm Emma", <다음 예시>],
        "good morning": ["Good morning, Sam", <다음 예시>]
        ]
    }},
    {{
        "단어":"hello",
        "뜻":"안녕",
        "유사어": [
        "hi": ["Hi, I'm Emma", <다음 예시>],
        "good morning": ["Good morning, Sam", <다음 예시>]
        ]
    }},
    {{
        "단어":"hello",
        "뜻":"안녕",
        "유사어": [
        "hi": ["Hi, I'm Emma", <다음 예시>],
        "good morning": ["Good morning, Sam", <다음 예시>]
        ]
    }}
    ]]

"""
    user_message = f"""
    노래 가사: {query}

"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
            "type": "json_object"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return json.loads(response.choices[0].message.content)


In [33]:
result = extract_eng_words(lyrics)

In [34]:
print(result)

{'json_list': [{'단어': 'ghost', '뜻': '유령', '유사어': {'phantom': ['The phantom appeared in the old castle.', 'He felt like a phantom in his own home.'], 'spirit': ['The spirit of the forest is said to protect the animals.', 'She has a free spirit, always traveling and exploring.']}}, {'단어': 'queen', '뜻': '여왕', '유사어': {'monarch': ['The monarch ruled the kingdom with wisdom.', 'As a monarch, she had many responsibilities.'], 'empress': ['The empress wore a crown of gold.', 'She was known as the empress of fashion.']}}, {'단어': 'believe', '뜻': '믿다', '유사어': {'trust': ['I trust you with my secrets.', 'Trust is the foundation of any relationship.'], 'faith': ['She has faith in her abilities.', 'Faith can move mountains.']}}, {'단어': 'shine', '뜻': '빛나다', '유사어': {'glow': ['The fireflies glow in the dark.', 'Her face glowed with happiness.'], 'sparkle': ['The stars sparkle in the night sky.', 'Her eyes sparkle with excitement.']}}, {'단어': 'moment', '뜻': '순간', '유사어': {'instant': ['In an instant, every

In [36]:
# print(result)

for word_dict in result['json_list']:
    print(f"단어: {word_dict['단어']}")
    print(f"뜻: {word_dict['뜻']}")
    
    print("유사어:")
    for sim_word in word_dict['유사어']:
        print(sim_word, ":", word_dict['유사어'][sim_word])
    print("\n")


단어: ghost
뜻: 유령
유사어:
phantom : ['The phantom appeared in the old castle.', 'He felt like a phantom in his own home.']
spirit : ['The spirit of the forest is said to protect the animals.', 'She has a free spirit, always traveling and exploring.']


단어: queen
뜻: 여왕
유사어:
monarch : ['The monarch ruled the kingdom with wisdom.', 'As a monarch, she had many responsibilities.']
empress : ['The empress wore a crown of gold.', 'She was known as the empress of fashion.']


단어: believe
뜻: 믿다
유사어:
trust : ['I trust you with my secrets.', 'Trust is the foundation of any relationship.']
faith : ['She has faith in her abilities.', 'Faith can move mountains.']


단어: shine
뜻: 빛나다
유사어:
glow : ['The fireflies glow in the dark.', 'Her face glowed with happiness.']
sparkle : ['The stars sparkle in the night sky.', 'Her eyes sparkle with excitement.']


단어: moment
뜻: 순간
유사어:
instant : ['In an instant, everything changed.', 'The instant he saw her, he knew she was special.']
second : ['Just give me a second 

### ReAct 기법

- Reason & Action 기법: 현재 상황에 대한 통찰 이후 다음 행동에 대한 작성을 유도하는 기법

In [37]:
##### 연애 코치

def dating_coach(query, temperature=0.3):
    client = OpenAI()

    system_instruction = """
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로서 사용자의 고민을 해결해주세요.

    ### 출력 형식 ###
    1. 상황 분석:

    2. 행동 계획:

    3. 실행:
    
    """

    user_message = f"""
    사용자의 현재 상황: {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1.0,
        frequency_penalty=1.0,
        presence_penalty=1.0
    )
    return response.choices[0].message.content

In [38]:
problem = "내 연인이 나를 버리고 싶어 한다. 나는 어떻게 해야 할까?"

print(dating_coach(problem))


    1. 상황 분석:
       - 연인이 당신을 떠나고 싶어하는 이유를 이해하는 것이 중요합니다. 이는 관계의 문제를 해결하고, 서로의 감정을 더 잘 이해할 수 있는 기회를 제공합니다.
       - 이별에 대한 두려움과 불안감은 자연스러운 반응이지만, 감정적으로 냉정함을 유지하며 상황을 객관적으로 바라보는 것이 필요합니다.

    2. 행동 계획:
       - 대화: 먼저 연인과 솔직한 대화를 나누세요. 그들이 왜 이런 결정을 고려하게 되었는지 직접 듣고, 그들의 입장을 존중하세요.
       - 자기 성찰: 자신의 행동이나 태도에서 개선할 점이 있는지 돌아보고, 이를 바탕으로 긍정적인 변화를 시도해 보세요.
       - 전문가 상담: 만약 혼자서 해결하기 어렵다면 커플 상담사와 같은 전문가의 도움을 받는 것도 좋은 방법입니다.

    3. 실행:
       - 진솔한 대화 시간을 마련하여 상대방에게 당신의 감정을 전달하고 그들의 생각을 경청하세요.
       - 자신에게 집중하면서 개인적인 성장 목표를 설정하고 이를 위해 노력하세요.
       - 필요한 경우 중립적인 제3자의 조언이나 지원을 받아 관계 회복에 힘쓰세요.

   이러한 접근 방식을 통해 여러분 모두가 원하는 방향으로 관계가 발전되기를 바랍니다.


##### 냉장고를 부탁해

 - query: 냉장고에 있는 재료 목록
 - return: query로 받은 재료를 활용한 레시피 작성
 - [hint] 현재 상황에 대한 분석/파악 > 행동 계획 > 실행

In [50]:
def finish_my_fridge(query, temperature=0.3):
    client = OpenAI()

    system_instruction = """
    역할 부여 : 당신은 냉장고를 부탁해 레시피 작성 선생님입니다.
    역할 정의 : 냉장고에 있는 재료를 활용한 레시피 작성, 레시피는 자취생들이 쉽게 준비할 수 있는 레시피로 작성
    타겟 정의 : 20, 30대 자취생들이 쉽게 준비할 수 있는 레시피로 작성

    ### 지시사항 ###
    1. 냉장고에 있는 재료를 파악하고 현재 상황에 대한 분석/파악
    2. 현재 냉장고에 있는 재료를 활용한 레시피 작성, 레시피는 자취생들이 쉽게 준비할 수 있는 레시피로 작성


    ### 출력 형식 ###
    1. 상황 분석:

    2. 행동 계획:

    3. 실행:
    """
    
    user_message = f"""
    냉장고에 있는 재료: {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1.0,
        frequency_penalty=1.0,
        presence_penalty=1.0
    )
    return response.choices[0].message.content
        

In [52]:
query = """돼지고기, 버섯, 김치, 치킨, 감자"""

print(finish_my_fridge(query))

1. 상황 분석:
   - 냉장고에 있는 재료는 돼지고기, 버섯, 김치, 치킨, 감자입니다.
   - 이 재료들을 활용하여 자취생들이 쉽게 만들 수 있는 간단한 요리를 계획할 필요가 있습니다.
   - 각 재료의 조합을 고려했을 때 한식과 양식을 결합한 퓨전 요리가 가능해 보입니다.

2. 행동 계획:
   - 주재료로 돼지고기를 사용하고 부재료로 김치와 버섯을 활용하여 '김치 돼지볶음'을 준비합니다.
   - 감자는 사이드 메뉴로 '감자버터구이'를 만듭니다.
   - 치킨은 이미 조리된 상태라면 별도로 추가적인 준비 없이 함께 곁들일 수 있습니다.

3. 실행:

**김치 돼지볶음 레시피**

- **재료**: 
  - 돼지고기 (200g)
  - 김치 (1컵)
  - 버섯 (100g)
  - 양파 (선택 사항으로 반 개 정도 있으면 좋습니다.)
  - 다진 마늘 (1큰술)
  - 고추장 또는 고춧가루 (옵션으로 매운맛 조절 가능)

- **조리 방법**:
  1. 팬에 기름을 두르고 다진 마늘과 얇게 썬 양파(있다면)를 넣어 볶아 향을 냅니다.
  2. 여기에 슬라이스한 돼지고기를 넣고 익혀줍니다.
  3. 고기가 어느 정도 익으면 잘게 썬 김치를 넣고 함께 볶습니다.
     *Tip: 신김치를 사용할 경우 설탕 약간 추가하면 맛이 부드러워집니다.*
  
4. 마지막으로 손질한 버섯과 고추장을 취향껏 첨가하고 모든 재료가 잘 섞이고 익도록 중불에서 볶아줍니다.

5. 완성된 요리는 밥 위에 올려 덮밥 형태로 즐길 수도 있고 따로 접시에 담아도 됩니다.


**감자버터구이 레시피**

- **재료**: 
    감자(중간 크기) 두 개
    소금 및 후추
    버터

- **조리 방법**:
    감자를 깨끗하게 씻어 껍질째 얇게 슬라이스 합니다.(두께 약0.5cm정도).
    
팬이나 오븐용기에 슬라이스 된 감자를 깔고 그 위에 소금과 후추를 뿌립니다.

작은 큐브 모양으로 자른 버터를 골고루 올린 뒤 예열된 오븐에서 구워주거나 팬에서 노릇노릇하게 

### 면접 질문 생성

- query: 채용 공고<br>

https://www.saramin.co.kr/zf_user/jobs/relay/view?isMypage=no&rec_idx=51963352&recommend_ids=eJxNj8ENxEAIA6u5PwaD4X2FpP8uLsopu%2Fsc2RpDujG97WrYR990JEVcY%2F6gBSNrp9Fq3og%2F3nXDTh2lo%2Byj7IUWNjGvGSPIaqsqEVxpt6b92HVDvmV0FrmHDA1hqSzrru%2BUQfK4Slmzd6HSQgxqeqkgOp4H8QP%2BKD%2Bl&view_type=search&searchword=AI엔지니어&searchType=search&gz=1&t_ref_content=generic&t_ref=search&relayNonce=507720b6b4ea0eb9e838&paid_fl=n&search_uuid=b5987049-cf6e-45cd-95d8-61c1033f20cc&immediately_apply_layer_open=n#seq=0

**과제**

In [ ]:
# 인성면접/기술면접으로 나누어 각각 예상 질문과 답변을 생성해 text로 반환
def job_interview(query, temperature=0.3):

In [ ]:
# hard_skill(기술 질문) / soft_skill(태도/생각 질문)으로 나누어 예상
질문-답변을 json형식으로 반환
def job_interview_json(query, temperature=0.3):